## Sliding windows and Classifiers
This is the final procedure of features engineerings. It should be done after the features extraction and after the one-hot-encoding procedure.<br>
The end of the notebook has an example of a some classifers which try to predict single porteins

### Some libraries

In [9]:
#Standard
import pandas as pd
import numpy as np
import time
import warnings
import matplotlib.pyplot as plt
#Sliding windows
from scipy import signal
#Module
from modules.feature_extraction import *
from modules.models import *
#Classifiers
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
#Metricsc
from sklearn import metrics
print('Done')

Done


### Importing DataFrame 'one-hot-enc'

In [2]:
#Import data
df = pd.read_csv(r'datasets\one-hot-enc.csv')
#Get data ready
LIP = df.LIP
df.drop(['LIP_SCORE', 'LIP'], inplace = True, axis = 1)
#Elimate NaN
df.REL_ASA[df.REL_ASA.isna()] = df.REL_ASA.mean()
df.head()

C:\Users\fgrim\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,PDB_ID,CHAIN_ID,RES_ID,REL_ASA,PHI,PSI,NH_O_1_relidx,NH_O_1_energy,O_NH_1_relidx,O_NH_1_energy,...,MC_SC,NO_EDGE_LOC,SC_MC,SC_SC,HBOND,IAC,NO_EDGE_TYPE,PIPISTACK,VDW,CHAIN_LEN
0,1cee,A,1,1.000000,360.0,97.6,0.0,0.0,2.0,-0.3,...,0,1,0,0,0,0,1,0,0,179
1,1cee,A,2,0.348485,-91.3,147.8,48.0,-0.1,50.0,-1.7,...,0,0,0,2,2,0,0,0,1,179
2,1cee,A,3,0.387324,-142.6,136.7,-2.0,-0.3,50.0,-0.2,...,0,1,0,0,0,0,1,0,0,179
3,1cee,A,4,0.005917,-94.3,154.4,48.0,-1.9,50.0,-1.6,...,0,0,0,3,2,0,0,0,3,179
4,1cee,A,5,0.346341,-112.5,70.5,-2.0,-0.2,71.0,-1.6,...,0,0,1,0,1,0,0,0,1,179


### Apply sliding windows

Use odd numbers for windows or some extra NaN could be generated

In [3]:
df_slided = sliding_windows(data=df, window=5, std=0.5, get_time=True, ignore_warnings=True)
df_slided.head()

16.904082775115967


,PDB_ID,CHAIN_ID,RES_ID,REL_ASA,PHI,PSI,NH_O_1_relidx,NH_O_1_energy,O_NH_1_relidx,O_NH_1_energy,...,MC_SC,NO_EDGE_LOC,SC_MC,SC_SC,HBOND,IAC,NO_EDGE_TYPE,PIPISTACK,VDW,CHAIN_LEN
0,1cee,A,1,0.219966,65.653323,27.239665,1.248171,-0.010854,3.113415,-0.111555,...,0.0,0.227134,0.000000,0.054268,0.054268,0.0,0.227134,0.0,0.027134,45.514025
1,1cee,A,2,0.107274,-12.391516,35.921342,9.548952,-0.028268,11.414196,-0.353654,...,0.0,0.054201,0.000000,0.400201,0.400134,0.0,0.054201,0.0,0.200201,45.514025
2,1cee,A,3,0.087148,-33.527040,35.530943,2.198303,-0.114148,12.711603,-0.129449,...,0.0,0.200067,0.000067,0.135335,0.108335,0.0,0.200067,0.0,0.108335,45.514025
3,1cee,A,4,0.021065,-25.776722,36.505443,9.498173,-0.393661,13.281823,-0.369036,...,0.0,0.027067,0.027067,0.600201,0.427335,0.0,0.027067,0.0,0.627201,45.514025
4,1cee,A,5,0.069454,-27.405446,21.215131,2.203000,-0.140235,16.914891,-0.444710,...,0.0,0.000067,0.200000,0.108268,0.308335,0.0,0.000067,0.0,0.308268,45.514025


### Model - Leave-One-Out of Protein
Here we run a cycle that for every protein of the dataset it use that protein as test set and the rest as training. <br>
We print Accuracy, Precision, Recall and Confusion Matrix

#### Build Classifier

In [31]:
clf = RandomForestClassifier(n_estimators=100)

True

#### Validation phase - LOOCV
Very long procedure for some classifiers. Use verbose to see results for each protein (advised if testing the model for the first time)

In [49]:
results = loo_cv(data = df_slided, clf=clf, 
                 target = LIP,
                 bad_condition=(0.75, 0.75),
                 ign_warnings = True, 
                 get_times = True, verbose = True)

Ieration number: 1
1cee
Accuracy: 0.962
Balanced Accuracy: 0.96
Precision: 0.865
Recall: 0.957
F1 score: 0.909
[[184   7]
 [  2  45]]
______________________________________________________
Ieration number: 2
1dev
Accuracy: 0.991
Balanced Accuracy: 0.976
Precision: 1.0
Recall: 0.951
F1 score: 0.975
[[194   0]
 [  2  39]]
______________________________________________________
Ieration number: 3
1dow
Accuracy: 0.996
Balanced Accuracy: 0.998
Precision: 0.969
Recall: 1.0
F1 score: 0.984
[[205   1]
 [  0  31]]
______________________________________________________


KeyboardInterrupt: 

#### Get some results of the model

In [46]:
print('The avarage balanced accuracy is {}\n'.format(results[1]))
print('The avarage f1 score is {}\n'.format(results[3]))
print('The proteins which gave bad results are:\n{}\n'.format(results[4]))
print('Number of proteins which gave bad results: {}\n'.format(len(results[4])))

The avarage balanced accuracy is 0.9268194444444444

The avarage f1 score is 0.8885694444444445

The proteins which gave bad results are:
[('1hrt', 0.535, 0.755, 3), ('1i7w', 1.0, 0.0, 3), ('1jsu', 0.768, 0.698, 3), ('1kil', 0.378, 0.578, 3), ('1rf8', 0.5, 0.0, 3), ('1sc5', 0.766, 0.622, 3), ('1sqq', 0.986, 0.719, 3), ('1ymh', 0.51, 0.76, 3), ('2a6q', 0.743, 0.922, 3), ('1ozs', 0.826, 0.667, 3), ('1j2x', 0.583, 0.348, 3)]

Number of proteins which gave bad results: 11



### Features Selection

In [45]:
if type(clf) == RandomForestClassifier    
    feat_imp = []
    for i,j in zip(clf.feature_importances_, df_slided.columns[3:]):
        feat_imp.append((i, j))

sorted(feat_imp, reverse=True)

[(0.6052153234603773, 'CHAIN_LEN'),
 (0.03647912555502266, 'O_NH_2_relidx'),
 (0.0355247945702235, 'NH_O_2_relidx'),
 (0.03531151712510219, 'REL_ASA'),
 (0.027422486239711025, 'NH_O_1_relidx'),
 (0.023877310842232595, 'O_NH_1_relidx'),
 (0.018055651940762463, 'O_NH_2_energy'),
 (0.018028979931704804, 'NH_O_1_energy'),
 (0.017808015813022937, 'NO_STRUCT'),
 (0.017424187135781647, 'O_NH_1_energy'),
 (0.01645820717885662, 'PSI'),
 (0.016062262768930552, 'PHI'),
 (0.015094670691089464, 'H_O_2_energy'),
 (0.006347434210165826, 'STRAND'),
 (0.006115353366598539, 'LYS'),
 (0.0059263098648442755, 'GLY'),
 (0.005856067543307142, 'LEU'),
 (0.005815857051626397, 'GLU'),
 (0.005521898554469201, 'SER'),
 (0.00541911500856324, 'ASP'),
 (0.004844090616303901, 'BEND'),
 (0.004814745669846882, 'ALA'),
 (0.004697157691440323, 'ALPHA_ELIX'),
 (0.004662874273091615, 'TURN'),
 (0.004464269566035991, 'GLN'),
 (0.0042833423583753935, 'ILE'),
 (0.004234627029897049, 'PRO'),
 (0.004230878540259504, 'ARG'),
 (0

### New DataSet with less features

In [58]:
low_feat = list(df_slided.columns[47:64])
df_new = df_slided.copy()
df_new.drop(low_feat, axis = 1, inplace = True)
df_new.head()

,PDB_ID,CHAIN_ID,RES_ID,REL_ASA,PHI,PSI,NH_O_1_relidx,NH_O_1_energy,O_NH_1_relidx,O_NH_1_energy,...,THR,TPO,TRP,TYR,VAL,10_ELIX,ALPHA_ELIX,BEND,ISOLATED_BETA_BRIGE,CHAIN_LEN
0,1cee,A,1,0.219966,65.653323,27.239665,1.248171,-0.010854,3.113415,-0.111555,...,0.027134,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,45.514025
1,1cee,A,2,0.107274,-12.391516,35.921342,9.548952,-0.028268,11.414196,-0.353654,...,0.027134,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,45.514025
2,1cee,A,3,0.087148,-33.527040,35.530943,2.198303,-0.114148,12.711603,-0.129449,...,0.200000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,45.514025
3,1cee,A,4,0.021065,-25.776722,36.505443,9.498173,-0.393661,13.281823,-0.369036,...,0.027067,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,45.514025
4,1cee,A,5,0.069454,-27.405446,21.215131,2.203000,-0.140235,16.914891,-0.444710,...,0.000067,0.0,0.0,0.0,0.000067,0.0,0.0,0.0,0.0,45.514025


### Re-test the model

In [59]:
results = loo_cv(data = df_new, clf=clf, 
                 target = LIP,
                 bad_condition=(0.75, 0.75),
                 ign_warnings = True, 
                 get_times = True, verbose = True)

Ieration number: 1
1cee
Accuracy: 0.954
Balanced Accuracy: 0.939
Precision: 0.86
Recall: 0.915
F1 score: 0.887
[[184   7]
 [  4  43]]
______________________________________________________
Ieration number: 2
1dev
Accuracy: 0.991
Balanced Accuracy: 0.976
Precision: 1.0
Recall: 0.951
F1 score: 0.975
[[194   0]
 [  2  39]]
______________________________________________________
Ieration number: 3
1dow
Accuracy: 1.0
Balanced Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 score: 1.0
[[206   0]
 [  0  31]]
______________________________________________________
Ieration number: 4
1fqj
Accuracy: 0.972
Balanced Accuracy: 0.985
Precision: 0.714
Recall: 1.0
F1 score: 0.833
[[321  10]
 [  0  25]]
______________________________________________________
Ieration number: 5
1g3j
Accuracy: 1.0
Balanced Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 score: 1.0
[[522   0]
 [  0  41]]
______________________________________________________
Ieration number: 6
1hrt
Accuracy: 0.615
Balanced Accuracy: 0.505
Precision:

Ieration number: 46
1tce
Accuracy: 0.967
Balanced Accuracy: 0.983
Precision: 0.6
Recall: 1.0
F1 score: 0.75
[[111   4]
 [  0   6]]
______________________________________________________
Ieration number: 47
1r1r
Accuracy: 0.997
Balanced Accuracy: 0.999
Precision: 0.875
Recall: 1.0
F1 score: 0.933
[[734   2]
 [  0  14]]
______________________________________________________
Ieration number: 48
1mxl
Accuracy: 0.981
Balanced Accuracy: 0.989
Precision: 0.857
Recall: 1.0
F1 score: 0.923
[[92  2]
 [ 0 12]]
______________________________________________________
Ieration number: 49
2fym
Accuracy: 1.0
Balanced Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 score: 1.0
[[858   0]
 [  0  15]]
______________________________________________________
Ieration number: 50
1iwq
Accuracy: 0.994
Balanced Accuracy: 0.972
Precision: 1.0
Recall: 0.944
F1 score: 0.971
[[139   0]
 [  1  17]]
______________________________________________________
Ieration number: 51
1fv1
Accuracy: 0.9
Balanced Accuracy: 0.8
Precisio

In [60]:
print('The avarage balanced accuracy is {}\n'.format(results[1]))
print('The avarage f1 score is {}\n'.format(results[3]))
print('The proteins which gave bad results are:\n{}\n'.format(results[4]))
print('Number of proteins which gave bad results: {}\n'.format(len(results[4])))

The avarage balanced accuracy is 0.9252361111111109

The avarage f1 score is 0.8897361111111112

The proteins which gave bad results are:
[('1hrt', 0.505, 0.752, 3), ('1i7w', 1.0, 0.0, 3), ('1jsu', 0.775, 0.71, 3), ('1kil', 0.382, 0.585, 3), ('1rf8', 0.5, 0.0, 3), ('1sc5', 0.766, 0.622, 3), ('1sqq', 0.986, 0.719, 3), ('1ymh', 0.522, 0.772, 3), ('2a6q', 0.681, 0.953, 3), ('1ozs', 0.876, 0.727, 3), ('1j2x', 0.556, 0.333, 3)]

Number of proteins which gave bad results: 11



### Try all classifiers (VERY LONG). Skip this part if you want to try a single model

In [12]:
#Create classifiers
classifiers = [
    KNeighborsClassifier(10),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=100),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

In [ ]:
for clf in classifiers:
    print('Classifier:\n{}:\n'.format(clf))
    #Get result
    results = loo_cv(data = df_slided, clf=clf, 
                     target = LIP,
                     bad_condition=(0.75, 0.75),
                     ign_warnings = True, 
                     get_times = True, verbose = False)
    #Print result
    print('The avarage balanced accuracy is {}\n'.format(results[1]))
    print('The avarage f1 score is {}\n'.format(results[3]))
    print('The balanced accuracies of every protein are:\n{}\n'.format(results[0]))
    print('The f1 scores of every protein are:\n{}\n'.format(results[2]))
    print('The proteins which gave bad results are:\n{}\n'.format(results[4]))
    print('Number of proteins which gave bad results: {}\n'.format(len(results[4])))
    print('--------------------------------------------------------------------------')
    print('--------------------------------------------------------------------------')

Classifier:
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='uniform'):

Time taken: 37.137951374053955

The avarage balanced accuracy is 0.9722222222222222

The avarage f1 score is 0.9166666666666666

The balanced accuracies of every protein are:
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

The f1 scores of every protein are:
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,